# <a id='toc1_'></a>[__Пайплайны обработки данных в задаче прогнозирования успеха по заявке на грант__](#toc0_)

**Содержание**<a id='toc0_'></a>    
- [__Пайплайны обработки данных в задаче прогнозирования успеха по заявке на грант__](#toc1_)    
  - [__Теория и примеры__](#toc1_1_)    
  - [__Импорты и настройки__](#toc1_2_)    
  - [__Фабула__](#toc1_3_)    
  - [__Обзор данных__](#toc1_4_)    
  - [__Подготовка данных: резка на обучение и тест, выделение вещественных и категориальных признаков__](#toc1_5_)    
  - [__Универсальная функция для различных сценариев обработки данных__](#toc1_6_)    
  - [__Кодирование категориальных признаков__](#toc1_7_)    
  - [__Заполнение пропущенных значений__](#toc1_8_)    
  - [__Масштабирование вещественных признаков__](#toc1_9_)    
  - [__Балансировка классов__](#toc1_10_)    
  - [__Стратификация выборки__](#toc1_11_)    
  - [__Полиномиальные признаки__](#toc1_12_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

***
## <a id='toc1_1_'></a>[__Теория и примеры__](#toc0_)

[__ml_logreg.ipynb__](https://github.com/EvgenyMeredelin/machine-learning-notes-and-codes/blob/main/ml_logreg/ml_logreg.ipynb)

***
## <a id='toc1_2_'></a>[__Импорты и настройки__](#toc0_)

In [1]:
# стандартная библиотека
import warnings
warnings.filterwarnings('ignore')

In [2]:
# сторонние библиотеки
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, PolynomialFeatures, StandardScaler
)
import numpy as np
import pandas as pd

***
## <a id='toc1_3_'></a>[__Фабула__](#toc0_)

В работе будут рассмотрены основные техники предобработки данных, применяемые для обучения модели логистической регрессии. По 38 признакам заявки на грант (область исследований, информация об академическом бэкграунде соискателей, размер гранта и т.д.) требуется предсказать, будет ли она одобрена. Датасет включает в себя информацию о 6 тыс. заявках на грант, поданные в университете Мельбурна в период 2004-2008 гг.

***
## <a id='toc1_4_'></a>[__Обзор данных__](#toc0_)

In [3]:
# первые пять строк датасета
df = pd.read_csv('unimelb.csv')
df.head()

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,...,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
0,1,21A,50A,A,230202.0,50.0,230203.0,30.0,230204.0,20.0,...,3098.0,31.0,Yes,>=0 to 5,2.0,0.0,0.0,4.0,2.0,0.0
1,1,4D,10A,D,320801.0,100.0,0.0,0.0,0.0,0.0,...,2553.0,25.0,Yes,>=0 to 5,3.0,1.0,0.0,2.0,0.0,0.0
2,0,NaN,NaN,NaN,320602.0,50.0,321004.0,30.0,321015.0,20.0,...,2813.0,25.0,NaN,Less than 0,1.0,5.0,0.0,7.0,2.0,0.0
3,0,51C,20C,A,291503.0,60.0,321402.0,40.0,0.0,0.0,...,2553.0,25.0,NaN,more than 15,2.0,1.0,5.0,6.0,9.0,1.0
4,0,24D,30B,NaN,380107.0,100.0,0.0,0.0,0.0,0.0,...,2923.0,25.0,NaN,Less than 0,0.0,2.0,0.0,0.0,0.0,0.0


In [4]:
# полнота данных и типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Grant.Status                            6000 non-null   int64  
 1   Sponsor.Code                            5387 non-null   object 
 2   Grant.Category.Code                     5387 non-null   object 
 3   Contract.Value.Band...see.note.A        3539 non-null   object 
 4   RFCD.Code.1                             5583 non-null   float64
 5   RFCD.Percentage.1                       5583 non-null   float64
 6   RFCD.Code.2                             5583 non-null   float64
 7   RFCD.Percentage.2                       5583 non-null   float64
 8   RFCD.Code.3                             5583 non-null   float64
 9   RFCD.Percentage.3                       5583 non-null   float64
 10  RFCD.Code.4                             5583 non-null   floa

***
## <a id='toc1_5_'></a>[__Подготовка данных: резка на обучение и тест, выделение вещественных и категориальных признаков__](#toc0_)

In [5]:
X = df.drop(columns='Grant.Status')
y = df['Grant.Status']  # целевая переменная

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0
)

In [6]:
# вещественные признаки
num_features = [
    'RFCD.Percentage.1', 'RFCD.Percentage.2', 'RFCD.Percentage.3', 
    'RFCD.Percentage.4', 'RFCD.Percentage.5',
    'SEO.Percentage.1', 'SEO.Percentage.2', 'SEO.Percentage.3',
    'SEO.Percentage.4', 'SEO.Percentage.5', 'Year.of.Birth.1', 
    'Number.of.Successful.Grant.1', 'Number.of.Unsuccessful.Grant.1'
]

# категориальные признаки
cat_features = list(set(X.columns) - set(num_features))

***
## <a id='toc1_6_'></a>[__Универсальная функция для различных сценариев обработки данных__](#toc0_)

Из свойств логистической модели следует, что:
* все предикторы должны быть числовыми, а в случае присутствия среди них категориальных их требуется некоторым способом преобразовать в вещественные числа;
* у предикторов не должно быть пропущенных значений, т.е. все пропущенные значения перед применением модели следует каким-то образом заполнить.

Поэтому базовым этапом в предобработке любого датасета для логистической регрессии будет кодирование категориальных признаков и удаление или интерпретация пропущенных значений (при наличии).

Сразу напишем универсальную функцию, которая реализует различные сценарии обработки данных, включая:
* заполнение пропущенных значений с возможностью выбора стратегии заполнения;

[__sklearn.impute.SimpleImputer__](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn-impute-simpleimputer)

> Univariate imputer for completing missing values with simple strategies.<br>
Replace missing values using a descriptive statistic (e.g. mean, median, or most frequent) along each column, or using a constant value. When `strategy` is "constant" and `fill_value` is None, `fill_value` will be 0 when imputing numerical data. 

* опцию добавления дополнительной обработки числовых признаков — например, масштабирования;

[__sklearn.preprocessing.StandardScaler__](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn-preprocessing-standardscaler)

> Standardize features by removing the mean and scaling to unit variance.

[__sklearn.preprocessing.PolynomialFeatures__](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html#sklearn-preprocessing-polynomialfeatures)

> Generate polynomial and interaction features.

* кодирование категориальных признаков;

[__sklearn.preprocessing.OneHotEncoder__](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn-preprocessing-onehotencoder)

> Encode categorical features as a one-hot numeric array.

* инициализацию классификатора заданными параметрами;

[__sklearn.linear_model.LogisticRegression__](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn-linear-model-logisticregression)

> Logistic Regression (aka logit, MaxEnt) classifier.

* оборачивание перечисленных выше шагов в пайплайн обработки;

[__sklearn.pipeline.Pipeline__](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn-pipeline-pipeline)

> Pipeline of transforms with a final estimator.

* поиск лучшей модели на кросс-валидации с перебором параметра регуляризации $C$ модели sklearn.linear_model.LogisticRegression;

[__sklearn.model_selection.GridSearchCV__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn-model-selection-gridsearchcv)

> Exhaustive search over specified parameter values for an estimator.

* для лучшей модели расчет метрики ROC AUC на тестовой выборке.

[__sklearn.metrics.roc_auc_score__](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn-metrics-roc-auc-score)

> Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

[__Column Transformer with Mixed Types__](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html#column-transformer-with-mixed-types)

In [7]:
def grid_search(strategy, extra_num_transformers=None, **clf_kwargs):
    """
    Универсальная функция для различных сценариев обработки данных.
    """
    num_transformer = Pipeline(  # пайплайн с вакантными слотами
        [('imputer', SimpleImputer(strategy=strategy))]
        + (extra_num_transformers or [])
    )
    preprocessor = ColumnTransformer([
        ('imputer_and_optional_extra', num_transformer, num_features),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ])
    steps = [
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(**clf_kwargs))
    ]
    search = GridSearchCV(
        Pipeline(steps),  # итоговый пайплайн с классификатором
        param_grid={'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]},
        cv=3,
        scoring='precision'
    )
    search.fit(X_train, y_train)
    y_pred = search.best_estimator_.predict_proba(X_test)[:,1]
    return roc_auc_score(y_test, y_pred)

***
## <a id='toc1_7_'></a>[__Кодирование категориальных признаков__](#toc0_)

Обязательный шаг, работает всегда.

***
## <a id='toc1_8_'></a>[__Заполнение пропущенных значений__](#toc0_)

Посчитаем ROC AUC для двух стратегий заполнения пропущенных значений: 
* заполнение средним значением признака (средним по столбцу), заполнение нулями.

In [8]:
clf_kwargs = dict(solver='liblinear')
scores = [grid_search(strategy, **clf_kwargs) for strategy in ('mean', 'constant')]
scores

[0.8854700421084674, 0.8842957566098414]

***
## <a id='toc1_9_'></a>[__Масштабирование вещественных признаков__](#toc0_)

* заполнение пропусков нулями
* масштабирование вещественных признаков

In [9]:
grid_search(
    strategy='constant', 
    extra_num_transformers=[('scaler', StandardScaler())],
    **clf_kwargs
)

0.885044379082622

***
## <a id='toc1_10_'></a>[__Балансировка классов__](#toc0_)

[__class_weight__](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn-linear-model-logisticregression)

> The "balanced" mode uses the values of `y` to automatically adjust weights inversely proportional to class frequencies in the input data.

* заполнение пропусков нулями
* масштабирование вещественных признаков
* балансировка классов

In [10]:
clf_kwargs['class_weight'] = 'balanced'

grid_search(
    strategy='constant', 
    extra_num_transformers=[('scaler', StandardScaler())],
    **clf_kwargs
)

0.887009358399606

***
## <a id='toc1_11_'></a>[__Стратификация выборки__](#toc0_)

[__stratify__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn-model-selection-train-test-split)

> If not None, data is split in a stratified fashion, using this as the class labels.

* стратификация `X` при разделении на обучение и тест (примерное сохранение в обучении и тесте исходного соотношения классов)
* заполнение пропусков нулями
* масштабирование вещественных признаков

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)

grid_search(
    strategy='constant', 
    extra_num_transformers=[('scaler', StandardScaler())],
    **clf_kwargs
)

0.8751318545718707

***
## <a id='toc1_12_'></a>[__Полиномиальные признаки__](#toc0_)

* стратификация `X` при разделении на обучение и тест (примерное сохранение в обучении и тесте исходного соотношения классов)
* заполнение пропусков нулями
* добавление в `X` полиномиальных признаков
* масштабирование вещественных признаков

In [12]:
clf_kwargs['fit_intercept'] = False

grid_search(
    strategy='constant', 
    extra_num_transformers=[
        ('poly', PolynomialFeatures()),
        ('scaler', StandardScaler())
    ],
    **clf_kwargs
)

0.8869913025739007

***